In this assignment we will look at a typical image based machine learning task.

## Image classification 

For this task the whole image is used to classify what's happening.

For this specific task, we will be trying to classify COVID-19 using pneumonia x-rays.  Please note, the literature has mostly suggested CT scans are not an effective way of figuring out what type of disease you have.  This exercise is for academic purposes _only_.

Steps:


1. Download the pneumonia data.  

You can find it here:

https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

move the folder to this directory and unzip it.  Please don't change any folder names or the below script will not work.  Also make sure the folder is in the same directory as this notebook!

2. load the pneumonia data into a dataframe:

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Check RAM usage
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [0]:
colab = True
download_data = False

if colab:
  from google.colab import drive
  drive.mount('/content/drive/')

  import os
  os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/Colab Notebooks/chest-xray-pneumonia/"

  #changing the working directory
  %cd /content/drive/My Drive/Colab Notebooks/chest-xray-pneumonia/

  from google.colab.patches import cv2_imshow # display image in colab

if download_data:
  # Dowload Kaggle dataset and unzip
  !kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
  !unzip \*.zip  && rm *.zip

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/chest-xray-pneumonia/'
/content


In [0]:
%cd /content/drive/My\ Drive/Tung

/content/drive/My Drive/Tung


In [0]:
# Import
import glob

from PIL import Image
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import skimage.transform
import skimage.color

import numpy as np
import cv2

import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

EPOCHS = 10 # Change this latter
BS = 8


def load_training_data():
    paths = [
        "chest_xray/train/NORMAL/*",
        "chest_xray/train/PNEUMONIA/*"
    ]
    labels = []
    image_paths = []
    for path in paths:
        for im_path in glob.glob(path):
            if path == "chest_xray/train/NORMAL/*":
                labels.append("NORMAL")
            if path == "chest_xray/train/PNEUMONIA/*":
                labels.append("PNEUMONIA")
            image_paths.append(im_path)
    return np.array(image_paths), np.array(labels)

def load_testing_data():
    paths = [
        "chest_xray/test/NORMAL/*",
        "chest_xray/test/PNEUMONIA/*"
    ]
    labels = []
    image_paths = []
    for path in paths:
        for im_path in glob.glob(path):
            if path == "chest_xray/test/NORMAL/*":
                labels.append("NORMAL")
            if path == "chest_xray/test/PNEUMONIA/*":
                labels.append("PNEUMONIA")
            image_paths.append(im_path)
    return np.array(image_paths), np.array(labels)

train_paths, train_labels = load_training_data()
test_paths, test_labels = load_testing_data()



# Change here for shorter time
runtime_lim = True
num_sample = 1000
if runtime_lim:
  import random
  random.seed(23)

  train_idx = random.choices(range(len(train_paths)), k=num_sample)
  test_idx = random.choices(range(len(test_paths)), k=num_sample)

  train_paths = train_paths[train_idx]
  train_labels = train_labels[train_idx]

  test_paths = test_paths[test_idx]
  test_labels = test_labels[test_idx]

3. read the data into memory, I recommend open-cv for this:

`python -m pip install opencv-python` 

if you don't already have it!

In [0]:
def load_images(image_paths):
    return [Image.open(p, mode='r') for p in image_paths]

train_images = load_images(train_paths) 
test_images = load_images(test_paths)

4. resize the images to a standard size - 

Note: it ought to be a box.  So the width and height should be the same size.

In [0]:
# VGG16 use 224x224
image_size = 224

def resize_and_greyscale(im):
  im = im.resize((image_size, image_size))
  return im.convert('L')

def resize_images(images):
    return [resize_and_greyscale(img) for img in images] 

train_images = resize_images(train_images)
test_images = resize_images(test_images)

5. Greyscale the images

In [0]:
def greyscale_images(images):
  pass
  # return [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]

# Comments out becasue:
# resize and greyscale to reduce memory and computation

# train_images = greyscale_images(train_images)
# test_images = greyscale_images(test_images)

6. prepare the data for training the model.

For this you'll need to transform the test and train image objects into a numpy array.

In [0]:
def to_rgb1(im):
  try:
    w, h = im.shape
  except:
    return im
  ret = np.empty((w, h, 3), dtype=np.uint8)
  ret[:, :, 0] = im
  ret[:, :, 1] = im
  ret[:, :, 2] = im
  return ret

def img_to_array(im):
  ret = np.array(im)
  ret = to_rgb1(ret)
  
  return ret

def features_to_np_array(images):
  return np.array([img_to_array(img) for img in images])
 
    

train_images = features_to_np_array(train_images)
test_images = features_to_np_array(test_images)

# # Rescale
train_images = train_images/255.0
test_images = test_images/255.0
# cv2_imshow(test_images[1])

# tmp = img_to_array(train_images[0])

Next you'll need to do the same for the labels:

Note: You'll need to apply the `to_categorical` function after transforming to a numpy array

In [0]:
from tensorflow.keras.utils import to_categorical

code_map = {"NORMAL": 0, "PNEUMONIA": 1, 0: "NORMAL", 1: "PNEUMONIA"}

def labels_to_np_array(labels):
    labels = [code_map[l] for l in labels]
    np.asarray(labels)
    return to_categorical(labels, num_classes=2)

train_labels = labels_to_np_array(train_labels)
test_labels = labels_to_np_array(test_labels)

7. Seperate into train and test with `train_test_split` from scikit-learn

In [0]:
# train test split code goes here
from sklearn.model_selection import train_test_split

X = np.concatenate((train_images, test_images), axis = 0)
y = np.concatenate((train_labels, test_labels), axis = 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=52)

8. Make the last four layers of VGG16 with imagenet weights trainable and then retrain the model.

To understand how to do this, please see the following tutorial:

https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/

In [0]:
# your model and training code goes here
from keras import models
from keras import layers
from keras import optimizers
from keras.applications import VGG16


#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# Only allow the last 4 layers trainable
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

# Create the model
model = models.Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)

# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

# Compule model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_5 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 514       
Total params: 21,137,986
Trainable params: 13,502,722
Non-trainable params: 7,635,264
_________________________________________________________________


In [0]:
# Model Train

model.fit(X_train, y_train, batch_size=BS, epochs=EPOCHS)

Epoch 1/10
1340/1340 [==============================] - 24s 18ms/step - loss: 4.9497 - acc: 0.6776
Epoch 2/10
1340/1340 [==============================] - 24s 18ms/step - loss: 4.9466 - acc: 0.6784
Epoch 3/10
1340/1340 [==============================] - 24s 18ms/step - loss: 4.9466 - acc: 0.6784
Epoch 4/10
1340/1340 [==============================] - 24s 18ms/step - loss: 4.9466 - acc: 0.6784
Epoch 5/10
1340/1340 [==============================] - 24s 18ms/step - loss: 4.9466 - acc: 0.6784
Epoch 6/10
1340/1340 [==============================] - 24s 18ms/step - loss: 4.9466 - acc: 0.6784
Epoch 7/10
1340/1340 [==============================] - 24s 18ms/step - loss: 4.9466 - acc: 0.6784
Epoch 8/10
1340/1340 [==============================] - 24s 18ms/step - loss: 4.9466 - acc: 0.6784
Epoch 9/10
1340/1340 [==============================] - 24s 18ms/step - loss: 4.9466 - acc: 0.6784
Epoch 10/10
1340/1340 [==============================] - 24s 18ms/step - loss: 4.9466 - acc: 0.6784


8. Check your score with classification_report from scikit-learn

Now that you've trained your model, call `model.predict` to get the predicted values for classification.  
Then compare your predicted values with y_test

In [0]:
def print_classification_report(y_test, y_pred):
  y_pred = y_pred > 0.5
  y_pred = np.argmax(y_pred, axis=1) 
  y_test = y_test > 0.5
  y_test = np.argmax(y_test, axis=1) 
  print(classification_report(y_test, y_pred, target_names=["NORMAL", "PNEUMONIA"]))

# classification report code goes here.
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

target_names = ["NORMAL", "PNEUMONIA"]
print_classification_report(y_test, y_pred)

              precision    recall  f1-score   support

      NORMAL       0.00      0.00      0.00       205
   PNEUMONIA       0.69      1.00      0.82       455

    accuracy                           0.69       660
   macro avg       0.34      0.50      0.41       660
weighted avg       0.48      0.69      0.56       660



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


9. Data augmentation

Now that you have a classifier, let's see if data augmentation improves things!  

You can use the `ImageDataGenerator` that comes with keras.  Here's how to import it:

`from tensorflow.keras.preprocessing.image import ImageDataGenerator`

Here's the documentation: https://keras.io/preprocessing/image/

Here's an example of it getting used in the wild, in case you get stuck:

https://www.pyimagesearch.com/2020/03/16/detecting-covid-19-in-x-ray-images-with-keras-tensorflow-and-deep-learning/

In [0]:
# augment your data here
from tensorflow.keras.preprocessing.image import ImageDataGenerator

augDataGen = ImageDataGenerator(
      rotation_range=20,
      fill_mode='nearest')

# Change these to according to RAM usage
train_batchsize = 100
val_batchsize = 10


trainGen = augDataGen.flow(X_train, y_train, batch_size=BS)


10. retrain your classifier

Now that you have augmented training data, please retrain your classifier.  The code should basically be the same.

In [0]:
# Create the model
aug_model = models.Sequential()

# new training code goes here

# Add the vgg convolutional base model
aug_model.add(vgg_conv)

# Add new layers
aug_model.add(layers.Flatten())
aug_model.add(layers.Dense(64, activation='relu'))
aug_model.add(layers.Dropout(0.5))
aug_model.add(layers.Dense(2, activation='softmax'))

# Compule model
opt = "adam"
aug_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])


aug_model.fit_generator(trainGen, steps_per_epoch=len(X_train)/BS,
                        validation_data=(X_test, y_test), validation_steps=len(X_test)/BS,
                        epochs=EPOCHS)

Epoch 1/10
168/167 [==============================] - 33s 196ms/step - loss: 10.3786 - acc: 0.3246 - val_loss: 10.5733 - val_acc: 0.3106
Epoch 2/10
168/167 [==============================] - 32s 193ms/step - loss: 10.4244 - acc: 0.3216 - val_loss: 10.5733 - val_acc: 0.3106
Epoch 3/10
168/167 [==============================] - 32s 193ms/step - loss: 6.6939 - acc: 0.5657 - val_loss: 4.7638 - val_acc: 0.6894
Epoch 4/10
168/167 [==============================] - 33s 194ms/step - loss: 5.5612 - acc: 0.6388 - val_loss: 4.7638 - val_acc: 0.6894
Epoch 5/10
168/167 [==============================] - 33s 194ms/step - loss: 5.6986 - acc: 0.6284 - val_loss: 4.7638 - val_acc: 0.6894
Epoch 6/10
168/167 [==============================] - 33s 194ms/step - loss: 5.7214 - acc: 0.6284 - val_loss: 4.7638 - val_acc: 0.6894
Epoch 7/10
168/167 [==============================] - 33s 194ms/step - loss: 6.0761 - acc: 0.6052 - val_loss: 10.5733 - val_acc: 0.3106
Epoch 8/10
168/167 [==============================

11. re-evaluate your classifier

Now that you've augmented the data, please re-evaluate your classifer.  Use classification report like before.

In [0]:
# classification report goes here
print(classification_report(y_test, y_pred))

12. Evaluate the difference with data augmentation and without:

Did things improve?  Did they stay the same?  Did they get worse?  Please try to come up with an explanation of why you got the results you did.

### Explanation of results go here

In [0]:
covid_model = aug_model

13. Getting COVID19 data

Now that you have a trained classifier with pneumonia, we are going to use this with COVID data.  

Clone this repo:

https://github.com/ieee8023/covid-chestxray-dataset

use the clone command: `git clone [REPO]`

to get the data locally.  

Make sure to run this command in the same folder as this jupyter notebook.

14. Read the data into memory

The set up for this data repository is a little different.  Please use the following code to read the data into memory:

In [0]:
import pandas as pd

def get_covid19():
    base = "covid-chestxray-dataset/"
    metadata = pd.read_csv(base+"metadata.csv")
    labels = []
    image_paths = []
    for index, row in metadata.iterrows():
        labels.append(row["finding"])
        image_paths.append(base+"images/"+row["filename"])
    return labels, image_paths

labels, covid_image_paths = get_covid19()

# Remove files with .gz in name
for idx, path in reversed(list(enumerate(covid_image_paths))):
  if ".gz" in path:
    labels.pop(idx)
    covid_image_paths.pop(idx)

15. preprocess images

you'll need to run the following functions on this data:

1. load_images
2. resize_images
3. greyscale_images
4. features_to_np_array
5. labels_to_np_array

Make sure to run each of those functions in order!

In [0]:
# label reduction code goes here
## Strip out labels other than 'No Finding' and 'COVID-19' from the dataset
## do this before load image

lop = ["No Finding", "COVID-19"]

for i, label in reversed(list(enumerate(labels))):
  if label not in lop:
    covid_image_paths.pop(i)
    labels.pop(i)

# convert to np array
covid_image_paths = np.array(covid_image_paths)
labels = np.array(labels)

if runtime_lim:
  covid_idx = random.choices(range(len(covid_image_paths)), k=num_sample)
  
  covid_image_paths = covid_image_paths[covid_idx]
  labels = labels[covid_idx]

In [0]:
# add your function calls to covid_image_paths here

def preprocess_image(img_path):
  img = Image.open(img_path, mode='r')
  img = img.convert('L')
  img = img.resize((image_size, image_size))
  img = img_to_array(img)

  return img/255.0

covid_image = np.array([preprocess_image(img_path) for img_path in covid_image_paths])

16. Strip out labels other than 'No Finding' and 'COVID-19' from the dataset

There are two straight forward ways to do this:

1) use a for-loop and keep track of indices

2) read labels and features into a dataframe and then filter to those two label types.  Your choice!

In [0]:
# label reduction code goes here

# Do this before 15

17. Predict on the new images

Here you'll use the classifier you trained on just pneumonia/not pneumonia to try and classify COVID-19 and no finding.  You'll use the pneumonia/not pneumonia classifier as a featurizer to do this.

Much of the code has been written, you'll just need to supply your trained classifier as input.

Please predict the labels from the classifier.  Then run `classification_report` to see how well your classifier did.

In [0]:
#prediction code goes here

import cv2
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import glob
import code

def extract_features_covid(model, width, height):
    base = "covid-chestxray-dataset/"
    metadata = pd.read_csv(base+"metadata.csv")
    labels = []
    feature_list = []
    image_paths = []
    for index, row in metadata[:20].iterrows():
        if row["finding"] == "COVID-19" and ".gz" not in row["filename"]:
            labels.append("COVID")

            im_path = base+"images/"+row["filename"]
            image_paths.append(im_path)

            img = np.expand_dims(preprocess_image(im_path), axis=0)
   
            features = model.predict(img)
            features_np = np.array(features)
            feature_list.append(features_np.flatten())

    return np.array(feature_list), labels

def extract_features_not_covid(model, width, height):
    feature_list = []
    labels = []
    image_paths = []
    paths = [
        "chest_xray/test/NORMAL/*",
        "chest_xray/test/PNEUMONIA/*",
        "chest_xray/train/NORMAL/*",
        "chest_xray/train/PNEUMONIA/*"
        
    ]
    for path in paths:
        for im_path in glob.glob(path)[:20]:
            if path == "chest_xray/train/NORMAL/*":
                labels.append("CLEAR TRAIN")
                image_paths.append(im_path)
            if path == "chest_xray/test/NORMAL/*":
                labels.append("CLEAR TEST")
                image_paths.append(im_path)
            if path == "chest_xray/train/PNEUMONIA/*":
                labels.append("PNEUMONIA")
                image_paths.append(im_path)
            # im = cv2.imread(im_path)
            # im = cv2.resize(im, (width, height))
      
      
            img = np.expand_dims(preprocess_image(im_path), axis=0)
            features = model.predict(img)
            features_np = np.array(features)
            feature_list.append(features_np.flatten())

    return np.array(feature_list), labels

# please make a copy of your tuned model and save it to variable:
untuned_model = covid_model

# please specify the width and height you used for the image preprocessing
width = image_size
height = image_size

covid_features, covid_labels = extract_features_covid(untuned_model, width, height)
non_covid_features, non_covid_labels = extract_features_not_covid(untuned_model, width, height)
features = np.concatenate([non_covid_features, covid_features])
labels = covid_labels + non_covid_labels
X_train = []
y_train = []
X_test = []
y_test = []
for index, label in enumerate(labels):
    if label == "CLEAR TRAIN":
        X_train.append(features[index])
        y_train.append(0)
    if label == "PNEUMONIA":
        X_train.append(features[index])
        y_train.append(1)
    if label == "COVID":
        X_test.append(features[index])
        y_test.append(1)
    if label == "CLEAR TEST":
        X_test.append(features[index])
        y_test.append(0)

logit_clf = LogisticRegression()
logit_clf.fit(X_train, y_train)
y_pred = logit_clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      1.00      0.83        20
           1       0.00      0.00      0.00         8

    accuracy                           0.71        28
   macro avg       0.36      0.50      0.42        28
weighted avg       0.51      0.71      0.60        28



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


18. Compare and contrast how the classifier did on Pneumonia versus COVID-19

Did it do as well?  Worse?  About the same?  What conclusions can you draw?

### Add your answers here!

Now that we've looked at a bunch of base cases, let's see if we can improve things by changing the model architecture.  We'll do this with a bunch of discrete steps

1. Change the number of trainable layers

Here you will make more of the layers trainable.  For this we are going to use cross validation to try and figure out which the optimal number of trainable layers.  Please us from the last 6 layers to one layer.  So your range should be:

```
trainable_range = [-6, -5, -4, -3, -2, -1]
```

Also, your X and y data should be the pneumonia data only.  Since that's what we trained on.  We should not assume we have access to the COVID data, except for testing, which will do later on.

Here's a blog post detailing how to set this up: https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

Note you'll need to set the number of trainable layers inside of `model_create` in order to make this tunable.  

Please report mean and standard deviation for accuracy.

In [0]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=52)

def new_model(activation='relu', optimizer='adam', dropout_rate=0.5, trainable_range=-4, num_nodes=64):
  keras.backend.clear_session()

  #Load the VGG model
  vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

  # Only allow the last x layers trainable
  for layer in vgg_conv.layers[:trainable_range]:
      layer.trainable = False

  # Create the model
  model = models.Sequential()

  # Add the vgg convolutional base model
  model.add(vgg_conv)

  # Add new layers
  model.add(layers.Flatten())
  model.add(layers.Dense(num_nodes, activation=activation))
  model.add(layers.Dropout(dropout_rate))
  model.add(layers.Dense(2, activation='softmax'))

  # Compule model
  model.compile(loss='binary_crossentropy',
                optimizer=optimizer,
                metrics=['acc'])
  
  return model

In [0]:
from sklearn.model_selection import StratifiedKFold

seed = 34
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)


grid_model = KerasClassifier(build_fn=new_model, batch_size=BS, epochs=EPOCHS, verbose=0)

trainable_range = [-6, -4, -2]
param_grid = dict(trainable_range=trainable_range)

grid = GridSearchCV(estimator=grid_model, param_grid=param_grid, n_jobs=1, verbose=0)

grid_res = grid.fit(X_train[:5], y_train[:5])

print("Best: %f using %s" % (grid_res.best_score_, grid_res.best_params_))

Best: 0.600000 using {'trainable_range': -4}


In [0]:
# Cross validation code goes here

best_trainable_range = grid_res.best_params_['trainable_range']

scores_lst=[]

y_train_label = np.argmax(y_train, axis=1)

for train, test in kfold.split(X_train, y_train_label):
  model = new_model(trainable_range = best_trainable_range)
  model.fit(X_train[train], y_train[train], batch_size=BS, epochs=EPOCHS)
  scores = model.evaluate(X_train[test], y_train[test])
  scores_lst.append(scores)

print(scores_lst)

Epoch 1/10
2144/2144 [==============================] - 11s 5ms/step - loss: 0.3951 - acc: 0.8540
Epoch 2/10
2144/2144 [==============================] - 11s 5ms/step - loss: 0.2898 - acc: 0.8750
Epoch 3/10
2144/2144 [==============================] - 11s 5ms/step - loss: 0.2926 - acc: 0.8708
Epoch 4/10
2144/2144 [==============================] - 11s 5ms/step - loss: 3.6210 - acc: 0.7043
Epoch 5/10
2144/2144 [==============================] - 11s 5ms/step - loss: 4.9925 - acc: 0.6754
Epoch 6/10
2144/2144 [==============================] - 11s 5ms/step - loss: 4.9925 - acc: 0.6754
Epoch 7/10
2144/2144 [==============================] - 11s 5ms/step - loss: 4.9925 - acc: 0.6754
Epoch 8/10
2144/2144 [==============================] - 11s 5ms/step - loss: 4.9925 - acc: 0.6754
Epoch 9/10
2144/2144 [==============================] - 11s 5ms/step - loss: 4.9925 - acc: 0.6754
Epoch 10/10
536/536 [==============================] - 2s 4ms/step
Epoch 1/10
2144/2144 [=============================

2. Analyze your results

Do you think that changing the number of tunable layers matters?  Does it improve classification accuracy enough to warrant changing the number of tunable layers?  

### Analysis and explanation go here

2. Tune over a layer activation function

Please set the number of tunable layers to 4 again.

Now we are going to make the layer activation tunable.  

To do this, please change the model_create function so that each layer has it's own tunable activation function.  Then run your new cross validation code.

In [0]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)


grid_model = KerasClassifier(build_fn=new_model, batch_size=BS, epochs=EPOCHS, verbose=0)

activation = ['relu', 'tanh', 'sigmoid', 'linear']
param_grid = dict(activation=activation)

grid = GridSearchCV(estimator=grid_model, param_grid=param_grid, n_jobs=1, verbose=0)

grid_res = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_res.best_score_, grid_res.best_params_))

Best: 0.719030 using {'activation': 'sigmoid'}


In [0]:
# cross validation code goes here

best_activation = grid_res.best_params_['activation']

scores=[]

y_train_label = np.argmax(y_train, axis=1)

for train, test in kfold.split(X_train, y_train_label):
  model = new_model(activation = best_activation)
  model.fit(X_train[train], y_train[train], batch_size=BS, epochs=EPOCHS)
  scores = model.evaluate(X_train[test], y_train[test])
  scores.append(scores)

print(scores)

Epoch 1/10
2144/2144 [==============================] - 11s 5ms/step - loss: 0.7535 - acc: 0.6590
Epoch 2/10
2144/2144 [==============================] - 11s 5ms/step - loss: 0.6948 - acc: 0.6115
Epoch 3/10
2144/2144 [==============================] - 11s 5ms/step - loss: 0.6622 - acc: 0.6507
Epoch 4/10
2144/2144 [==============================] - 11s 5ms/step - loss: 0.6492 - acc: 0.6632
Epoch 5/10
2144/2144 [==============================] - 11s 5ms/step - loss: 0.6402 - acc: 0.6693
Epoch 6/10
2144/2144 [==============================] - 11s 5ms/step - loss: 0.6386 - acc: 0.6749
Epoch 7/10
2144/2144 [==============================] - 11s 5ms/step - loss: 0.6351 - acc: 0.6749
Epoch 8/10
2144/2144 [==============================] - 11s 5ms/step - loss: 0.6389 - acc: 0.6754
Epoch 9/10
2144/2144 [==============================] - 11s 5ms/step - loss: 0.6354 - acc: 0.6749
Epoch 10/10
536/536 [==============================] - 2s 3ms/step
Epoch 1/10
2144/2144 [=============================

3. Analyze your results

Does your choice of activation function matter?  When does the activation function perform best?  

Things to consider:

* Specifically does choosing the same activation function for all of the layers do best? 
* Does choosing different activation functions for each of the layers do best?
* Are they all within the same approximate accuracy range?
* do things vary wildly?

### Analysis and explanation go here

4. Tune over more hyperparameters

Now that we've tuned the activation functions, let's try tuning more parameters.  This time add tuning for the following parameters:

* number of neurons per layer
* weight initialization
* optimizer
* weight constraint
* activation function
* learning rate

Here is a great post on the range of values you should consider: https://www.wandb.com/articles/fundamentals-of-neural-networks

Here is some code that is also useful: https://www.kaggle.com/lavanyashukla01/training-a-neural-network-start-here

for understanding this practically.

In [0]:
activation='relu', 
optimizer= ['SGD', 'adam']
dropout_rates=[0.0, 0.2, 0.4, 0.6]
num_nodes=[16, 32, 64, 128]


param_grid = dict(activation=activation
                  optimizer=optimizer,
                  dropout_rate=dropout_rates,
                  num_nodes=num_nodes)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_res = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_res.best_score_, grid_res.best_params_))

SyntaxError: ignored

In [0]:
# cross validation code goes here

# cross validation code goes here

best_activation = grid_res.best_params_['activation']
best_optimizer = grid_res.best_params_['optimizer']
best_dropout_rates = grid_res.best_params_['dropout_rates']
best_num_nodes = grid_res.best_params_['num_nodes']

scores=[]

y_train_label = np.argmax(y_train, axis=1)

for train, test in kfold.split(X_train, y_train_label):
  model = new_model(activation = best_activation, optimizer=best_optimizer, 
                    dropout_rates = best_dropout_rates, num_nodes = best_num_nodes)
  
  model.fit(X_train[train], y_train[train], batch_size=BS, epochs=EPOCHS)
  scores = model.evaluate(X_train[test], y_train[test])
  scores.append(scores)

print(scores)

5. Tune over data augmentation

Here you'll take the best hyperparameters from your neural network, with 4 trainable layers, and then add them to a pipeline.  We will then tune over data augmentation parameters.  Report out your mean and standard deviation of accuracy.

Here we will create a scikit-learn pipline:

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

If you need an example with gridsearch and pipeline:

https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html

As a reminder, here is the documentation for data augmentation:

https://keras.io/preprocessing/image/

In [0]:
#cross validation code goes here

6. Analyze your results

Now that you've tuned over model parameters and preprocessing, what has a bigger impact?  Why do you think that might be the case?

### Analysis and explanation goes here

7. Using your best model and preprocessing to train a new model

Now you should select the best hyperparameters for the neural network and the best hyperparameters for the preprocesser and then combine them into a scikit-learn pipeline.  Next train a classifier with these new tuned hyperparameters.

In [0]:
#classifer generation code goes here

8. Let's see if things improved - time for `classification_report`

Now that you've tuned your model, let's see how well it does on our test set!  First call predict on the test data to get a prediction.  Then use `classification_report` to see how well the model does.

In [0]:
# prediction code goes here

9. Analsis and comparison

Now that you've seen how well your classifier does when it's been tuned, compare this with your previous model, that was untuned.  Are the precision, recall and f1-scores substantially different?  Why or why not?

### Analysis and explanation goes here

10. Prediction on COVID binary classification task with tuned model

Now you'll use your tuned classifier to try and predict on the binary COVID19 case.  Please change the model to your tuned model!

In [0]:
#prediction code goes here

import cv2
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import glob
import code

def extract_features_covid(model, width, height):
    base = "covid-chestxray-dataset/"
    metadata = pd.read_csv(base+"metadata.csv")
    labels = []
    image_paths = []
    for index, row in metadata.iterrows():
        if row["finding"] == "COVID-19":
            labels.append("COVID")
            image_paths.append(base+row["filename"])
            im = cv2.imread(im_path)
            im = cv2.resize(im, (width, height))
            features = model.predict(img)
            features_np = np.array(features)
            feature_list.append(features_np.flatten())

    return np.array(feature_list), labels

def extract_features_not_covid(model, width, height):
    feature_list = []
    labels = []
    paths = [
        "chest_xray/test/NORMAL/*",
        "chest_xray/test/PNEUMONIA/*",
        "chest_xray/train/NORMAL/*",
        "chest_xray/train/PNEUMONIA/*"
        
    ]
    for path in paths:
        for im_path in glob.glob(path):
            if path == "chest_xray/train/NORMAL/*":
                labels.append("CLEAR TRAIN")
            if path == "chest_xray/test/NORMAL/*":
                labels.append("CLEAR TEST")
            if path == "chest_xray/train/PNEUMONIA/*":
                labels.append("PNEUMONIA")
            im = cv2.imread(im_path)
            im = cv2.resize(im, (width, height))
            features = model.predict(img)
            features_np = np.array(features)
            feature_list.append(features_np.flatten())

    return np.array(feature_list), labels

# please make a copy of your tuned model and save it to variable:
# tuned_model = [YOUR MODEL NAME GOES HERE]

# please specify the width and height you used for the image preprocessing
# width = [YOUR WIDTH GOES HERE]
# height = [YOUR HEIGHT GOES HERE]

covid_features, covid_labels = extract_features_covid(tuned_model, width, height)
non_covid_features, non_covid_labels = extract_features_not_covid(tuned_model, width, height)
features = covid_features + non_covid_features
labels = covid_labels + non_covid_labels
X_train = []
y_train = []
X_test = []
y_test = []
for index, label in enumerate(labels):
    if label == "CLEAR TRAIN":
        X_train.append(features[index])
        y_train.append(0)
    if label == "PNEUMONIA":
        X_train.append(features[index])
        y_train.append(1)
    if label == "COVID":
        X_test.append(features[index])
        y_test.append(1)
    if label == "CLEAR TEST":
        X_test.append(features[index])
        y_test.append(0)

logit_clf = LogisticRegression()
logit_clf.fit(X_train, y_train)
y_pred = logit_clf.predict(X_test)
print(classification_report(y_test, y_pred))

11. Analyze your results

Now that you've seen the results of your tuned model, compare those with the results of the untuned model.  Did things get better? Worse?  Why do you think this may or may not be the case?

### Analysis of your results goes here